# Лекция 8: метаклассы, тестирование

## Метаклассы

Metaclasses are deeper magic than 99% of users should
never worry about. If you wonder whether you need them,
you don’t (the people who actually need them know with
certainty that they need them, and don’t need an explanation
about why).

by Tim Peters


## Классы и классы классов

Помним, что в Python всё является объектом:

In [1]:
print type(1)

<type 'int'>


In [2]:
class MyInt(int):
    def __add__(self, other):
        print "specializing addition"
        return super(MyInt, self).__add__(other)

i = MyInt(2)
print i + 2

specializing addition
4


Классы также являются объектами.

In [3]:
class Empty(object):
    pass

Воспользуемся встроенной функцией type() для определения типа объекта:

In [4]:
obj = Empty()
print type(obj)

<class '__main__.Empty'>


Проверим тип класса:

In [5]:
type(Empty)

type

Oh, wait... What if?

In [6]:
print type(tuple), type(list), type(int), type(float)

<type 'type'> <type 'type'> <type 'type'> <type 'type'>


* Классы сами по себе являются объектами типа type.  
* Это верно для всех new-style классов. 

Hmmm...

In [7]:
print type(type)

<type 'type'>


* type является предком себя - такую циклическую зависимость нельзя сделать в чистом Python, поэтом она реализуется  небольшим хаком в реализации Python.
* type имеет несколько значений в Python:
  * type() - как функция возвращая тип переданного ей объекта.
  * type - как конструктор типов.
  * type - как корневой тип для метаклассов, предок всех классов (подробности ниже).

## Метапрограммирование

* В общем случае, программы которые порождают другие программы или динамически модифицируются.
* В частном случае, возможность динамического создания новых типов во время выполнения.

Мы можем написать функцию, динамически создающую объекты:

In [8]:
def int_factory(s):
    i = int(s)
    return i

i = int_factory('100')
print i

100


Это простейший пример функции-фабрики, которая принимает некоторые аргументы, конструирует нужный объект и возвращает его.

* Ничего не мешает нам создать функцию, которая будет создавать объект типа type (т.е. некоторый класс) и возвращать его в качестве результата
* Такая функция называется метафункцией.

In [9]:
def class_factory():
    class Foo(object):
        pass
    return Foo

F = class_factory()
f = F()
print type(f)

<class '__main__.Foo'>


* Получившаяся функция создаёт класс, однако выглядит несколько странно.  
* Было бы отлично избежать явного написания кода и выполнять эти действия более динамически.
* Это можно сделать создавая наш класс напрямую от класса type.

In [10]:
def class_factory():
    return type('Foo', (), {})

F = class_factory()
f = F()
print(type(f))

<class '__main__.Foo'>


Вообще, запись вида:

In [11]:
class Empty(object):
    pass

Эквивалентна следующей конструкции:

In [12]:
Empty = type('Empty', (), {})

### type - конструктор типов

type(name, bases, dct)

* name - имя для создаваемого класса.
* bases - tuple задающий родительские классы для конструируемого класса.
* dct - словарь аттрибутов и методов создаваемого класса.

Следующие примеры дадут идентичные результаты:

In [13]:
class Foo(object):
    i = 4

class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print b.get_i()

4


In [14]:
Foo = type('Foo', (), dict(i=4))

Bar = type('Bar', (Foo,), dict(get_i = lambda self: self.i))

b = Bar()
print b.get_i()

4


### type как корневой тип для метаклассов

Мы можем создать собственный метакласс отнаследовавшись от type, а затем использовать его вместо type при создании классов.

При создании нового класса с помощью ключевого слова class происходит следующее:  
* Python выполняет тело внутри конструкции class как обычный блок кода.
* Получившийся namespace (словарь) содержит в себе аттрибуты будущего класса.
* Метакласс определяется путём поиска поля \_\_metaclass\_\_ в создаваемом классе, его предках (заданные метаклассы наследуются) или глобальной переменной \_\_metaclass\_\_.
* Найденный метакласс вызывается, ему передаётся имя, список базовых классов и аттрибуты создаваемого класса.

### Пример метакласса - регистрируем подклассы

Следующий метакласс просто добавляет словарь registry (реестр) и каждый раз при создании нового класса кладёт в registry имя этого класса.

In [15]:
class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls
            
        super(DBInterfaceMeta, cls).__init__(name, bases, dct)

Мы можем задать классу другой (вместо type) метакласс путём присваивания его специальному полю \_\_metaclass\_\_:

In [16]:
class DBInterface(object):
    __metaclass__ = DBInterfaceMeta
    
print(DBInterface.registry)

{}


После добавления подклассов:

In [17]:
class FirstInterface(DBInterface):
    pass

class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print(DBInterface.registry)

{'firstinterface': <class '__main__.FirstInterface'>, 'secondinterface': <class '__main__.SecondInterface'>, 'secondinterfacemodified': <class '__main__.SecondInterfaceModified'>}


### Составной пример метакласса

В Python существует два метода, которые используются при создании объекта (в т.ч. классов и метаклассов):  
* \_\_new\_\_ - конструктор, вызывается непосредственно для создания объекта, редко когда нужен (если нам надо иметь  контроль над созданием объекта).
* \_\_init\_\_ - инициализатор, вызывается после создания объекта для его инициализации, в основном используется этот метод.

В случае с метаклассами редко, но может иметь смысл перегрузка функции \_\_call\_\_ в метаклассе (вызывается, когда идёт создание объекта с помощью имени класса MyClass()).

По сути, эти три функции позволяют влиять на разные этапы создания класса и его объектов.

In [18]:
def make_hook(f):
    """Decorator to turn 'foo' method into '__foo__'"""
    f.is_hook = 1
    return f

class MyType(type):
    def __new__(cls, name, bases, attrs):

        if name.startswith('None'):
            return None

        # Go over attributes and see if they should be renamed.
        newattrs = {}
        for attrname, attrvalue in attrs.iteritems():
            if getattr(attrvalue, 'is_hook', 0):
                newattrs['__%s__' % attrname] = attrvalue
            else:
                newattrs[attrname] = attrvalue

        return super(MyType, cls).__new__(cls, name, bases, newattrs)

    def __init__(self, name, bases, attrs):
        super(MyType, self).__init__(name, bases, attrs)

        print "__init__ for {}".format(self)

    def __add__(self, other):
        class AutoClass(self, other):
            pass
        return AutoClass
        # Alternatively, to autogenerate the classname as well as the class:
        # return type(self.__name__ + other.__name__, (self, other), {})

    def say_hello(self):
        print "Hello, {}".format(self)

In [19]:
class MyObject:
    __metaclass__ = MyType

__init__ for <class '__main__.MyObject'>


In [20]:
class NoneSample(MyObject):
    pass

# Will print "NoneType None"
print type(NoneSample), repr(NoneSample)

<type 'NoneType'> None


In [21]:
class Example(MyObject):
    def __init__(self, value):
        self.value = value

    @make_hook
    def add(self, other):
        return self.__class__(self.value + other.value)

Example.say_hello()

__init__ for <class '__main__.Example'>
Hello, <class '__main__.Example'>


In [22]:
inst = Example(10)

print inst + inst

In [23]:
class Sibling(MyObject):
    pass

ExampleSibling = Example + Sibling
# ExampleSibling is now a subclass of both Example and Sibling (with no
# content of its own) although it will believe it's called 'AutoClass'
print ExampleSibling
print ExampleSibling.__mro__

__init__ for <class '__main__.Sibling'>
__init__ for <class '__main__.AutoClass'>
<class '__main__.AutoClass'>
(<class '__main__.AutoClass'>, <class '__main__.Example'>, <class '__main__.Sibling'>, <class '__main__.MyObject'>, <type 'object'>)


### Дополнительное чтение про метаклассы

* http://eli.thegreenplace.net/2011/08/14/python-metaclasses-by-example - местами более подробная версия этой лекции, с дополнительными примерами кода и применения в библиотеках.
* http://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/ - подробная статья, в деталях соединяющая (с картинками) материал этой и прошлой лекции.  

## Тестирование и Python

### Почему нужно писать тесты

* Python - динамически типизированный язык программирования, нет привычной по другим языкам статической проверки типов.
* Удобно авматоматизировать ручные проверки старых и новых функций и их взаимодействия.
* При внесении изменений в код хотелось бы иметь простой способ быстро убедиться, что ничего не сломал.
* При работе в команде это способ отслеживания состояния и работоспособности проекта.
* Тесты - некоторого рода документация, описывающая как взаимодействовать с кодом.

### Юнит-тестирование

* Существует ряд различных типов тестирования, которые применяются так или  иначе для проверки отдельных аспектов работы программы.
* Нас интересует один из часто используемых - юнит-тестирование.
* Юнит-тестирование (unit testing) предполагает написание тестов для небольших частей кода (функций или классов).
* В таких тестах проверяется выполнение ожидаемых инвариантов работы с помощью различных вариантов функции assert.
* Проверяем: что вернулось ожидаемое значение, что два значения равны, что было выброшено (или не выброшено) исключение,  что вызов завершился до истечения таймаута и т.п.

### Python unittest

* В результате развития подходом и библиотек для языка Java в какой-то момент появилась успешная библиотека JUnit.
* unittest - реализации идей этой библиотеки для языка Python.

Основные концепции выполнения тестов:

* создание начальных условий для отдельного теста
* задание завершающих действий для отдельного теста
* объединение тестов в группы
* автоматизированный запуск наборов тестов
* независимое отображение результатов выполнения тестов

Для реализации этих концепций используются понятия:

* Отдельный тест (test case, класс TestCase) - описывает действия подготовки и завершения (setUp и tearDown), а также сам тест.
* Набор тестов (test suite, TestSuite) - объединяет в себе несколько тестов.
* Тестовые установки (test fixture, TestFixture) - представляют действия для подготовки и завершения одного или нескольких тестов (создание временных таблиц, директорий и т.п.)
* Исполнитель тестов (test runner, TestRunner) - компонент, отвечающий за запуск тестов и отображения их результатов (текстовы, графический интерфейсы).

Пример отдельного теста:

In [24]:
import unittest

class TestStringMethods(unittest.TestCase):

  def test_upper(self):
      self.assertEqual('foo'.upper(), 'FOO')

  def test_isupper(self):
      self.assertTrue('FOO'.isupper())
      self.assertFalse('Foo'.isupper())

  def test_split(self):
      s = 'hello world'
      self.assertEqual(s.split(), ['hello', 'world'])
      # check that s.split fails when the separator is not a string
      with self.assertRaises(TypeError):
          s.split(2)

In [25]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestStringMethods)
unittest.TextTestRunner(verbosity=2).run(suite)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

Предлагается самостоятельно почитать подробнее, изучить примеры и попробовать писать тесты:

https://docs.python.org/2/library/unittest.html - официальная документация по библиотеке unittest

### Библиотеки для юнит-тестов

Существует также ряд других библиотек, которые могут быть интересны в будущем:

* doctest - выполнение тестов особого вида, помещённых прямо в docstring.
* py.test - альтернатива стандартному unittest, может подкупить простым интерфейсом.
* nose - расширяет unittest упрощая тестирование (test discovery, показатель test coverage).
* tox - автоматизация управления тестовым окружением, позволяет задавать матрицы параметров для тестов.
* mock - библиотека для тестирования, позволяет просто создавать объекты-заглушки для тестирования и заменять части системы на время тестирования (см. mock тестирование).

### Общие советы по написанию тестов

* Обычно тесты кладут в папку tests в проекте.
* Удобно использовать по одному файлу с тестами на каждый модуль.
* Стараться избавляться от избыточной дупликации кода в отдельных тестах, если это можно сделать.
* Отдельный тест должен проверять небольшую часть функциональности (если пройден, то она верна).
* Каждый тест должен быть полностью независимым.
* Тесты должны работать быстро, иначе будут проблемы с их частым и массовым запуском.
* Удобно, когда на каждый коммит в репозиторий автоматически запускаются тесты.
* Если есть долгие тесты, то можно их запускать по расписанию, а не при каждом коммите.
* Полезно писать как можно подробные имена тестов.